# Predicting energy production of windmills using weather forecasts

This project proposes a method of modelling weather forecast data with the express purpose of predicting the amount of energy produced by windmills. We will be using energy data from windmills located in Orkney, Scotland aswell as weather forecast data from the region, specifying windspeeds and directions at a given timestep. 

The windmills in Orkney are in charge of the majority of the energy production in the region, with over 500 windmills. Orkney being a cluster of islands off of the northern coast of Scotland, are subject to a lot of heavy wind making it an ideal place for windmills. In periods of incredibly strong wind and low local demand for energy, the network of windmills in Orkney produce an excess of energy which can be sold off to energy companies. 

Selling off spare energy is an active descision which requires knowing when excess energy will be generated and when to stop, as to not be subjected to fees and tarifs. This makes the ability to correctly predict when energy generation will be high an important fiscal tool. 

We therefore propose to use a varying array of supervised methods of regression to predict future energy generation. 

In [52]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn import svm
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge


from windTransformer import WindVectorTransformer
from windTransformer import WindDegreeTransformer
from windData import WindDataCollector


import numpy as np
import math
import pandas as pd

from influxdb import InfluxDBClient # install via "pip install influxdb"

import datetime

In [53]:
def eval_metrics(actual, pred):
	rmse = np.sqrt(mean_squared_error(actual, pred))
	mae = mean_absolute_error(actual, pred)
	r2 = r2_score(actual, pred)
	return rmse, mae, r2


In [54]:
time_two_years = (datetime.datetime.utcnow() - datetime.timedelta(days=365*2)).strftime("'%Y-%m-%dT%H:%M:%SZ'")

In [55]:
dataCollector = WindDataCollector()

gen_df = dataCollector.getGenerationData(now = time_two_years, delta="90")
wind_df = dataCollector.getWindData(now = time_two_years, delta="90")

gen_df_alligned = pd.merge_asof(wind_df,gen_df,left_index=True, right_index=True)

In [56]:
gen_df_alligned

,Direction,Lead_hours,Source_time,Speed,ANM,Non-ANM,Total
time,,,,,,,
2020-10-04 12:00:00+00:00,NE,1,1601802000,8.04672,11.368998,16.521,27.889998
2020-10-04 15:00:00+00:00,ENE,1,1601812800,8.04672,7.322000,11.403,18.725000
2020-10-04 18:00:00+00:00,NE,1,1601823600,7.15264,3.505000,8.177,11.682000
2020-10-04 21:00:00+00:00,ENE,1,1601834400,8.04672,3.505000,7.622,11.127000
2020-10-05 00:00:00+00:00,E,1,1601845200,7.15264,1.595000,4.931,6.526000
...,...,...,...,...,...,...,...
2021-01-01 21:00:00+00:00,NNW,1,1609527600,11.17600,4.887000,13.972,18.859000
2021-01-02 00:00:00+00:00,NNE,1,1609538400,8.04672,5.492000,15.204,20.696000
2021-01-02 03:00:00+00:00,NNW,1,1609549200,9.83488,5.371000,9.294,14.665000


# Data

The direction is the wind direction, so when it says S it means the wind is coming from the South

Speed is the wind speed in meters per second

The total is the energy generated at the timestep in megawatts

In [40]:
import plotly.express as px


fig = px.scatter(gen_df_alligned, x="Speed", y="Total")
fig.show()

In [57]:
w = WindDegreeTransformer()
gen_df_alligned["Direction_degree"] = gen_df_alligned["Direction"].apply(lambda x: w.md_dict[x])
gen_df_alligned[["Direction_degree","Speed","Total"]].corr()

,Direction_degree,Speed,Total
Direction_degree,1.000000,-0.180966,-0.164934
Speed,-0.180966,1.000000,0.812091
Total,-0.164934,0.812091,1.000000


In [58]:
train_length = int(len(gen_df_alligned)*0.9)

train_X = gen_df_alligned.iloc[:train_length][[
#    "Direction",
    "Speed"]]
test_X = gen_df_alligned.iloc[train_length:][[
#    "Direction",
    "Speed"]]

train_y = gen_df_alligned.iloc[:train_length]["Total"]
test_y = gen_df_alligned.iloc[train_length:]["Total"]


## Linear regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [59]:
pipeline = Pipeline(steps=[
    ("column_transformers", ColumnTransformer([
#        ("direction_degree_transformer", WindDegreeTransformer(), ["Direction"]),
        ("polynomial_features", PolynomialFeatures() , ["Speed"])
    ])),
	("linear_model", LinearRegression())
])
parameters = {'column_transformers__polynomial_features__degree':[1,2,3,4,5,6,7,8]}

In [60]:
tscv = TimeSeriesSplit(n_splits=5)
pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

pipeline.fit(train_X, np.ravel(train_y))

bestParams = pipeline.best_params_

predicted_qualities = pipeline.predict(test_X)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print(bestParams)
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

{'column_transformers__polynomial_features__degree': 4}
  RMSE: 4.146382232286991
  MAE: 3.3975975872837076
  R2: 0.8911320914049711


## Ridge Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [61]:
pipeline = Pipeline(steps=[
    ("column_transformers", ColumnTransformer([
#        ("direction_degree_transformer", WindDegreeTransformer(), ["Direction"]),
        ("polynomial_features", PolynomialFeatures() , ["Speed"])
    ])),
	("linear_model", Ridge())
])
parameters = {'column_transformers__polynomial_features__degree':[1,2,3,4,5,6,7,8]}

In [62]:
tscv = TimeSeriesSplit(n_splits=5)
pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

pipeline.fit(train_X, np.ravel(train_y))

bestParams = pipeline.best_params_

predicted_qualities = pipeline.predict(test_X)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print(bestParams)
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

{'column_transformers__polynomial_features__degree': 4}
  RMSE: 4.152158550659015
  MAE: 3.404418876602769
  R2: 0.8908285527082392


## Support Vector Machines CAAP

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [63]:
pipeline = Pipeline(steps=[
#	("WindVector_transform",WindVectorTransformer()),    
    ("column_transformers", ColumnTransformer([
#        ("direction_degree_transformer", WindDegreeTransformer(), ["Direction"]),
        ("polynomial_features", PolynomialFeatures() , ["Speed"])
    ])),
	("svm_model", svm.SVR())
])
parameters = {
    'svm_model__kernel':["rbf", "linear"],
    'svm_model__C':[1.0],
    'svm_model__gamma':[0.01, 0.1, 0.2, 0.5],
#    'column_transformers__polynomial_features__degree':[1,2,3,4,5,6,7,8]
}

In [64]:
tscv = TimeSeriesSplit(n_splits=5)
pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

pipeline.fit(train_X, np.ravel(train_y))

bestParams = pipeline.best_params_

predicted_qualities = pipeline.predict(test_X)

In [65]:
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("SVR model (gamma={}, kernel={}, C={})".format(bestParams["svm_model__gamma"], bestParams["svm_model__kernel"], bestParams["svm_model__C"]))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)


SVR model (gamma=0.01, kernel=rbf, C=1.0)
  RMSE: 4.2547868250427126
  MAE: 3.510660634091436
  R2: 0.8853651091283182


## MLP

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [21]:
pipeline = Pipeline(steps=[
	("WindVector_transform",WindVectorTransformer()),
	("MLP_model", MLPRegressor())
])
parameters = {
    'MLP_model__activation':['identity', 'logistic', 'tanh', 'relu'],
    'MLP_model__solver':['lbfgs', 'sgd', 'adam'],
    'MLP_model__learning_rate': ["constant"],
    'MLP_model__learning_rate_init':[0.001, 0.01, 0.1, 0.2, 0.5],
    'MLP_model__max_iter':[10, 50, 100, 200, 500]
}

In [22]:
tscv = TimeSeriesSplit(n_splits=5)
pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

pipeline.fit(train_X, np.ravel(train_y))

bestParams = pipeline.best_params_

predicted_qualities = pipeline.predict(test_X)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print(bestParams)
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').